In [1]:
import pandas as pd
import numpy as np
import cv2
import os
from tqdm import tqdm
from joblib import Parallel, delayed
from tqdm import tqdm


In [2]:
IMG_DIR = '../datasets/exports/uint8_png/'
LABEL_DIR = '../datasets/roi_det/detect/yolox_nano_bre_416_datav2/'
SAVE_DIR = '../datasets/crops/uint8_png@yolox_nano_bre_416_datav2/'
DTYPE = 'uint8'

In [3]:
os.makedirs(SAVE_DIR, exist_ok = True)

In [4]:
img_names = os.listdir(IMG_DIR)
    
def process_one(img_name):
    label_name = img_name.replace('.png', '.txt')
    img_path = os.path.join(IMG_DIR, img_name)
    label_path = os.path.join(LABEL_DIR, label_name)
    save_crop_path = os.path.join(SAVE_DIR, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_ANYDEPTH)
    if DTYPE == 'uint8':
        assert img.dtype == np.uint8
    elif DTYPE == 'uint16':
        assert img.dtype == np.uint16
    else:
        raise ValueError()
    
    img_h, img_w = img.shape[:2]
    
    try:
        label = np.loadtxt(label_path, delimiter= ' ').reshape((1, 5))
    except Exception as e:
        print('Error:', img_name)
        raise e
    
    xywh = label[0, 1:]
    ltrb = xywh.copy()
    ltrb[:2] = ltrb[:2] - ltrb[2:] / 2
    ltrb[2:] = ltrb[:2] + ltrb[2:]
    
    ltrb[ltrb > 1.0] = 1.0
    ltrb[ltrb < 0.0] = 0.0
    l = int(ltrb[0] * img_w)
    t = int(ltrb[1] * img_h)
    r = int(ltrb[2] * img_w)
    b = int(ltrb[3] * img_h)
    crop = img[t:b, l:r]
    try:
        cv2.imwrite(save_crop_path, crop)
    except:
        print(img_name)
        print(l, t, r, b)
        print(img_w, img_h)
        print(crop)
        

        
_ = Parallel(n_jobs=32, backend='multiprocessing')(
            delayed(process_one)(img_name)
            for img_name in tqdm(img_names))

100%|█████████████████████████████████████████████████████| 54706/54706 [1:15:08<00:00, 12.13it/s]
